In [1]:
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import ast
from collections import Counter

In [2]:
# !!! Modify this to match the name of the csv file of the patients you generated
file_path = "llm_patients_043025.csv"

In [3]:
# =============================================================================
# Hard-coded Demographic Distributions
# =============================================================================

""" This data is the same as the demographic distributions from the llm_patient_creator.ipynb file. """
# split distributions to a .py file?

# Age distribution (averaged for men & women).
AGE_WEIGHTS_MEN = np.array([0.2979, 0.1916, 0.1557, 0.2266, 0.1282])
AGE_WEIGHTS_WOMEN = np.array([0.2861, 0.1683, 0.1483, 0.2343, 0.1630])

# Relationship status percentages by age group: (18-25, 26-35, 36-45, 46-60, 60+)
RELATIONSHIP_STATUS_DISTRIBUTIONS = [ # Single, Long-term relationship, Married/Common-Law, Divorced/Separated, Widowded
    [69.26, 23.09,  7.40,  0.25,  0.00],  # 18-25
    [31.73, 10.58, 55.35,  2.15,  0.20],  # 26-35
    [10.88,  3.63, 76.00,  8.15,  1.35],  # 36-45
    [ 9.00,  3.00, 68.33, 15.33,  8.17],  # 46-60
    [11.25,  3.75, 54.17,  8.83, 33.50],  # 60+
]

# Probabilities of being "in a family" & "with children," by 4 age bins (0-24, 25-34, 35-44, 45-54).
IN_FAMILY = [
    [0.0756, 0.0701],  # female, male (0-24)
    [0.5102, 0.4765],  # female, male (25-34)
    [0.8340, 0.8178],  # female, male (35-44)
    [0.8426, 0.8557],  # female, male (45+)
]
WITH_CHILDREN = [
    [0.6997, 0.3003],  # yes, no  (0-24)
    [0.5193, 0.4807],  # yes, no  (25-34)
    [0.2056, 0.7944],  # yes, no  (35-44)
    [0.1771, 0.8229],  # yes, no  (45+)
]

SEX_WEIGHTS = [0.51, 0.49]
SEX_LABELS = ["Male", "Female"]

AGE_ORDER = ["18-25", "26-35", "36-45", "46-60", "60+"]

RELATIONSHIP_ORDER = ["Single", "Long-term relationship",
                          "Married/Common-Law", "Divorced/Separated", "Widowded"]


DISABILITY_CATEGORIES = [
        "Mental-health related", "Pain-related", "Seeing",
        "Learning", "Memory", "Mobility", "Flexibility",
        "Hearing", "Dexterity", "Developmental"
    ]

ETHNICITY_PROBABILITY = [0.0243, 0.0334, 0.4313, 0.2328, 0.1417, 0.0763, 0.0198, 0.0158, 0.0251]

ETHNICITY_ORDER = ["Indigenous", "Middle Eastern", "European", 
                   "East Asian", "South Asian", "Southeast Asian", 
                   "Latin American", "African", "Other"]

ETHNICITY_CATEGORIES = {
    "Indigenous": [
        "Indigenous"
    ],
    "Middle Eastern": ["Persian - Iran", "Hebrew - Israel"],
    "European": [
        "Czech - Czech Republic", "Danish - Denmark", "German - Austria", "German - Switzerland", "German - Germany",
        "Greek - Greece", "English - Canada", "English - United States", "English - United Kingdom", "English - Ireland",
        "Spanish - Spain", "Finnish - Finland", "French - Canada", "French - Switzerland", "French - France",
        "Croatian - Croatia", "Hungarian - Hungary", "Armenian - Armenia", "Italian - Italy", "Dutch - Belgium",
        "Dutch - Netherlands", "Norwegian - Norway", "Polish - Poland", "Portuguese - Portugal", "Romanian - Romania",
        "Russian - Russia", "Slovak - Slovakia", "Swedish - Sweden", "Ukrainian - Ukraine"
    ],
    "East Asian": [
        "Japanese - Japan", "Korean - South Korea",
        "Chinese - China", "Chinese - Taiwan"
    ],
    "South Asian": [
        "English - India", "Hindi - India", "Nepali - Nepal",
        "Tamil - India", "Bengali - Bangladesh", "English - Bangladesh",
        "Georgian - Georgia", "Azerbaijani - Azerbaijan"
    ],
    "Southeast Asian": [
        "English - Malaysia", "English - Philippines", "Filipino - Philippines",
        "Indonesian - Indonesia", "Thai - Thailand", "Tagalog - Philippines",
        "Vietnamese - Vietnam"
    ],
    "Latin American": [
        "Spanish - Argentina", "Spanish - Chile", "Spanish - Colombia",
        "Spanish - Mexico", "Portuguese - Brazil"
    ],
    "African": ["Zulu - South Africa"],
    "Other": ["English - Australia", "English - New Zealand"],
}

In [4]:
# =============================================================================
# Statistical Tests
# =============================================================================

def binomial_p_value(n, p, observed_count, tail='right'):
    """
    Compute the p-value for a binomial distribution using a normal approximation.
    """
    expected = n * p
    print("expected:", expected)
    sigma = np.sqrt(n * p * (1 - p))
    print("sigma:", sigma)
    print("obs:", observed_count)
    z_score = (observed_count - expected) / sigma
    print("z:", z_score)
    
    if observed_count > expected:
        print("right")
        print("observed:", observed_count)
        print("expected:", expected)
        p_value = 1 - stats.norm.cdf(z_score)
    else:  # left tail
        print("left")
        p_value = stats.norm.cdf(z_score)
    
    return z_score, p_value

def multinomial_p_value(observed_counts, expected_probs):
    """
    Compute the p-value for a multinomial distribution using the chi-square test.
    """
    total_n = sum(observed_counts)
    expected_counts = np.array(expected_probs) * total_n
    print("expected:", expected_counts)
    print("observed:", observed_counts)
    chi_square_stat, p_value = stats.chisquare(f_obs=observed_counts, f_exp=expected_counts)
    # chi_square_stat = 0
    # p_value = 0
    return chi_square_stat, p_value

In [5]:
# =============================================================================
# Analysis and Plotting
# =============================================================================

def analyze(df, column, expected_probs, title, chart_type='bar', order=None):
    """
    Perform statistical analysis for a given categorical column.
    """

    # If an order is provided, reindex the observed counts accordingly
    if order is not None:
        observed_counts = df[column].value_counts().reindex(order, fill_value=0)
    else:
        observed_counts = df[column].value_counts().sort_index()
        
    print("counts:", observed_counts)
    print(type(observed_counts))
    observed_probs = observed_counts / observed_counts.sum()
    print("probs:", observed_probs)
    
    # Normalize probabilities
    expected_probs = np.array(expected_probs) / np.sum(expected_probs)
    
    if len(expected_probs) == 2:
        # Binomial test
        print(observed_counts.sum(), expected_probs[1], observed_counts.iloc[1])
        z_score, p_value = binomial_p_value(observed_counts.sum(), expected_probs[1], observed_counts.iloc[1])
        print(f"{title} - Z-score: {z_score:.2f}, P-value: {p_value:.4f}")
    else:
        # Multinomial test
        chi_square_stat, p_value = multinomial_p_value(observed_counts.values, expected_probs)
        print(f"{title} - Chi-Square: {chi_square_stat:.2f}, P-value: {p_value:.4f}")
    

In [6]:
# =============================================================================
# Helper Functions
# =============================================================================

def categorize_ethnicity(ethnicity):
    for category, values in ETHNICITY_CATEGORIES.items():
         if ethnicity in values:
            return category
    return "Uncategorized"


def compute_expected_disability(row: pd.Series) -> pd.Series:
    """
    Hard-coded probability of being disabled by (sex, age),
    then distribute among 10 disability types.
    """

    # Hard-coded distribution across 10 types
    s = 38.6 + 61.8 + 27.4 + 20.7 + 18.2 + 39.2 + 40.3 + 20.7 + 18.4 + 5.7
    DISABILITY_PROBABILITY_TYPES_NORM = [
        38.6/s, 61.8/s, 27.4/s, 20.7/s, 18.2/s,
        39.2/s, 40.3/s, 20.7/s, 18.4/s, 5.7/s
    ]
    expected_values = {
        cat: p_type
        for cat, p_type in zip(DISABILITY_CATEGORIES, DISABILITY_PROBABILITY_TYPES_NORM)
    }
    return pd.Series(expected_values)

def categorize_age(age):
    """
    Categorize age into groups:
    18-25, 26-35, 36-45, 46-60, 60+.
    """
    if age < 18:
        return "Under 18"
    elif 18 <= age <= 25:
        return "18-25"
    elif 26 <= age <= 35:
        return "26-35"
    elif 36 <= age <= 45:
        return "36-45"
    elif 46 <= age <= 60:
        return "46-60"
    else:
        return "60+"

def compute_expected_age_probs(df):
    """
    Compute overall expected age probabilities using provided weights and the sex distribution in df.
    """
    
    # Calculate proportion of males and females in the dataset
    sex_counts = df['Sex'].value_counts(normalize=True)
    prop_male = sex_counts.get("Male", 0)
    prop_female = sex_counts.get("Female", 0)
    
    overall_expected = prop_male * AGE_WEIGHTS_MEN + prop_female * AGE_WEIGHTS_WOMEN
    print("Expected age probabilities:", overall_expected)
    return overall_expected, AGE_ORDER

In [7]:
def main():
    
    df = pd.read_csv(file_path, delimiter = "|")
    
    print("\n=== Statistical Tests and Graphs ===\n")
    
    # == Sex Analysis ==
    analyze(df, 'Sex', SEX_WEIGHTS, "Sex Distribution")

    # == Age Analysis== 
    df['Age_Category'] = df['Age'].apply(categorize_age)
    expected_age_probs, age_groups = compute_expected_age_probs(df)
    # Ensure the Age_Category is ordered correctly
    df['Age_Category'] = pd.Categorical(df['Age_Category'], categories=age_groups, ordered=True)
    analyze(df, 'Age_Category', expected_age_probs, "Age Distribution", order=age_groups)

    # == Ethnicity Analysis ==
    df["Ethnicity_Category"] = df["Ethnicity"].apply(categorize_ethnicity)
    analyze(df, 'Ethnicity_Category', ETHNICITY_PROBABILITY, "Ethnicity Distribution", order=ETHNICITY_ORDER)
    
    # == Disability Analysis ==
    df_yes_disability = df[df["Disabled"] == "Yes"]
    # 1) Parse and flatten the disability type lists
    all_types = []

    for val in df_yes_disability["Disability"]:
        if isinstance(val, str):
            try:
                parsed = ast.literal_eval(val)  # safely converts "['mobility', 'vision']" → list
                if isinstance(parsed, list):
                    all_types.extend(parsed)
            except:
                continue
        elif isinstance(val, list):
            all_types.extend(val)
            
    # 2) Count frequencies
    type_counts = Counter(all_types)
    # print("type counts:", type_counts)
    # total_obs = df_yes_disability.shape[0]
    total_obs = sum(type_counts.values())
    print("total obs:", total_obs)

    # 3) Build observed distribution dict
    observed_disability = {
        cat: type_counts.get(cat, 0) / total_obs
        for cat in DISABILITY_CATEGORIES
    }

    expected_disability_df = df_yes_disability.apply(compute_expected_disability, axis=1)
    expected_disability_avg = expected_disability_df.mean().to_dict()

    categories = list(expected_disability_avg.keys())

    observed_counts = pd.Series(type_counts)
    observed_counts = observed_counts.reindex(DISABILITY_CATEGORIES)
    print("counts:" , observed_counts)
    expected_counts = np.array([expected_disability_avg[cat] for cat in categories])

    expected_probs = np.array(expected_counts) / np.sum(expected_counts)

    # observed_counts = df_yes_disability['Disability'].value_counts().reindex(order, fill_value=0)

    title="Disability Breakdown"
    chi_square_stat, p_value = multinomial_p_value(observed_counts.values, expected_probs)
    print(f"{title} - Chi-Square: {chi_square_stat:.2f}, P-value: {p_value:.4f}")

    # Rename columns
    df.rename(columns={
        "Relationship Status": "Relationship_Status",
        "Children": "Has_Children",
    }, inplace=True)

    # Simplify relationship statuses
    relationship_mapping = {
        "Common-Law": "Married/Common-Law",
        "Married": "Married/Common-Law",
        "Divorced": "Divorced/Separated",
        "Separated": "Divorced/Separated"
    }
    df["Relationship_Status"] = df["Relationship_Status"].replace(relationship_mapping)
    df["Relationship_Status"] = df["Relationship_Status"].astype("category")


    # Convert "Has_Children" to yes/no
    df["Has_Children"] = df["Has_Children"].apply(
        lambda x: "Yes" if x != 0 else "No"
    )

    # 1) Compute observed fractions using crosstab (normalized by row)
    # df['Age_Category'] = df['Age'].apply(categorize_age)
    observed_rel_df = pd.crosstab(df["Age_Category"],
                                  df["Relationship_Status"],
                                  normalize="index")
    observed_rel_df = observed_rel_df.reindex(index=AGE_ORDER,
                                              columns=RELATIONSHIP_ORDER)

    # 2) Build expected distribution from hard-coded RELATIONSHIP_STATUS_DISTRIBUTIONS
    columns_for_rel = ["Single", "Long-term Relationship", "Married/Common-law",
                       "Divorced/Separated", "Widowed"]
    rel_df = pd.DataFrame(RELATIONSHIP_STATUS_DISTRIBUTIONS,
                          index=AGE_ORDER,
                          columns=columns_for_rel)

    # 3) Rename columns to match observed naming
    col_map = {
        "Long-term Relationship": "Long-term relationship",
        "Married/Common-law": "Married/Common-Law",
        "Widowed": "Widowded"
    }
    rel_df.rename(columns=col_map, inplace=True)
    expected_rel_df = rel_df.reindex(index=AGE_ORDER, columns=RELATIONSHIP_ORDER)

    # 4) Convert from percentages (0-100) to fractions (0-1)
    observed_matrix = observed_rel_df.values
    expected_matrix = expected_rel_df.values / 100.0

    # Convert the matrix form back into DataFrame (for the split-diagonal function)
    obs_df_rel = pd.DataFrame(observed_matrix,
                              index=AGE_ORDER,
                              columns=RELATIONSHIP_ORDER)
    exp_df_rel = pd.DataFrame(expected_matrix,
                              index=AGE_ORDER,
                              columns=RELATIONSHIP_ORDER)

    print("obs:", obs_df_rel)
    print("exp:", exp_df_rel)

    
    counts_rel_df = (
        pd.crosstab(df["Age_Category"], df["Relationship_Status"])
          .reindex(index=obs_df_rel.index,
                   columns=obs_df_rel.columns,
                   fill_value=0)
        )


    for age in obs_df_rel.index:
        total_n    = counts_rel_df.loc[age].sum()
        obs_counts = obs_df_rel.loc[age].values * total_n
        exp_probs  = exp_df_rel.loc[age].values

        # compute expected counts and re-normalize
        exp_counts = exp_probs * total_n
        # exp_counts = exp_counts / exp_counts.sum() * total_n

        # drop any zero-expected bins
        mask = exp_counts > 0
        oc = obs_counts[mask]
        ec = exp_counts[mask]

        chi2, p = stats.chisquare(f_obs=oc, f_exp=ec, sum_check = False)
        print(f"{age} → χ² = {chi2:.2f}, p = {p:.3f}")


    # 1) compute the total-N for each age group
    total_n = counts_rel_df.sum(axis=1)     # a Series indexed by age

    # 2) get the observed counts matrix: age × status
    obs_matrix = obs_df_rel.mul(total_n, axis=0).values

    # 3) get the expected counts matrix: age × status
    exp_matrix = exp_df_rel.mul(total_n, axis=0).values
    
    # 4) flatten to 1d
    f_obs = obs_matrix.flatten()
    f_exp = exp_matrix.flatten()

    # 6) renormalize expected counts to match the observed total exactly
    # f_exp = f_exp / f_exp.sum() * f_obs.sum()

    # 5) drop any zero‐expected bins (to avoid division-by-zero)
    mask = f_exp > 0
    f_obs = f_obs[mask]
    f_exp = f_exp[mask]

    # 7) run χ²
    chi2_all, p_all = stats.chisquare(f_obs=f_obs, f_exp=f_exp, sum_check=False)
    print("Combined χ² = %.2f, p = %.3f" % (chi2_all, p_all))


    # 1) Use the entire DataFrame
    df_for_children = df.copy()  # or df_for_children = df

    # 2) Define the new bins and labels: [0,25), [25,35), [35,45), [45,∞)
    age_bins = [0, 25, 35, 45, float('inf')]
    age_labels = ['0-24', '25-34', '35-44', '45+']

    df_for_children['Age_Group'] = pd.cut(
        df_for_children['Age'],
        bins=age_bins,
        labels=age_labels,
        right=False
    )

    # 3) Compute observed fraction with children (pivot into 2D table)
    observed_counts = (
        df_for_children.groupby(['Age_Group', 'Sex'])['Has_Children']
                       .value_counts(normalize=True)
                       .unstack(fill_value=0)
    )
    observed_df_children = (
        observed_counts[['Yes']]  # we only need the "Yes" column
        .reset_index()
        .pivot(index='Age_Group', columns='Sex', values='Yes')
    )

    # 4) Build the expected fraction with children
    expected_dict_children = {}
    for i, label in enumerate(age_labels):
        # Probability = in_family[i][sex_idx] * with_children[i][0]
        female_prob = IN_FAMILY[i][0] * WITH_CHILDREN[i][1]
        male_prob   = IN_FAMILY[i][1] * WITH_CHILDREN[i][1]
        expected_dict_children[(label, "Female")] = female_prob
        expected_dict_children[(label, "Male")]   = male_prob

    expected_series_children = pd.Series(expected_dict_children)
    expected_df_children = expected_series_children.unstack()
    
    results = []
    for age in expected_df_children.index:            # e.g. '0-24', '25-34',...
        # 1) subset to this age group
        sub = df_for_children[df_for_children['Age_Group'] == age]
        n   = len(sub)
        k   = sub['Has_Children'].eq('Yes').sum()

        # 2) build weighted expected probability p0 for this age:
        #    take each sex’s expected p and weight by how many of that sex are in this age
        counts_by_sex = sub['Sex'].value_counts().reindex(expected_df_children.columns, fill_value=0)
        p_by_sex      = expected_df_children.loc[age]   # a Series: index ['Female','Male']
        p0 = (counts_by_sex * p_by_sex).sum() / counts_by_sex.sum()

        # 3) call your helper
        z_stat, p_val = binomial_p_value(n, p0, k)

        results.append({
            'Age_Group': age,
            'n'        : n,
            'k'        : k,
            'exp_p0'   : p0,
            'z_stat'   : z_stat,
            'p_value'  : p_val
        })

    age_df = pd.DataFrame(results).set_index('Age_Group')
    print(age_df[['z_stat','p_value']])

    # 1) total N and total k
    n_tot = len(df_for_children)
    k_tot = df_for_children['Has_Children'].eq('Yes').sum()

    # 2) get counts per (Age_Group, Sex)
    counts = df_for_children.groupby(['Age_Group','Sex']).size()

    # 3) get expected p per cell as a Series with the same MultiIndex
    #    expected_df_children: index=Age_Group, columns=Sex
    exp_p = expected_df_children.stack()  # Series indexed by (Age_Group,Sex)

    # 4) expected total count
    E_tot = (counts * exp_p).sum()

    # 5) overall expected probability
    p_tot = E_tot / n_tot

    # 6) get overall z and p-value
    z_overall, p_overall = binomial_p_value(n_tot, p_tot, k_tot)

    print(f"Overall child–prob: z = {z_overall:.4f}, p = {p_overall:.4f}")
    
if __name__ == "__main__":
    main()


=== Statistical Tests and Graphs ===

counts: Sex
Female    500
Male      500
Name: count, dtype: int64
<class 'pandas.core.series.Series'>
probs: Sex
Female    0.5
Male      0.5
Name: count, dtype: float64
1000 0.49 500
expected: 490.0
sigma: 15.8082257068907
obs: 500
z: 0.6325820611000681
right
observed: 500
expected: 490.0
Sex Distribution - Z-score: 0.63, P-value: 0.2635
Expected age probabilities: [0.292   0.17995 0.152   0.23045 0.1456 ]
counts: Age_Category
18-25    313
26-35    170
36-45    145
46-60    230
60+      142
Name: count, dtype: int64
<class 'pandas.core.series.Series'>
probs: Age_Category
18-25    0.313
26-35    0.170
36-45    0.145
46-60    0.230
60+      0.142
Name: count, dtype: float64
expected: [292.   179.95 152.   230.45 145.6 ]
observed: [313 170 145 230 142]
Age Distribution - Chi-Square: 2.47, P-value: 0.6495
counts: Ethnicity_Category
Indigenous          25
Middle Eastern      29
European           396
East Asian         255
South Asian        132
Southe

/tmp/ipykernel_2536855/4138863116.py:198: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_for_children.groupby(['Age_Group', 'Sex'])['Has_Children']
/tmp/ipykernel_2536855/4138863116.py:253: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  counts = df_for_children.groupby(['Age_Group','Sex']).size()


ValueError: cannot join with no overlapping index names